<a href="https://colab.research.google.com/github/guumaster/colab-nbs/blob/master/books/deep-learning-with-python-notebooks/chapter11_part01_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparing the IMDB movie reviews data 

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup
!cat aclImdb/train/pos/4077_10.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  8813k      0  0:00:09  0:00:09 --:--:-- 15.8M
I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drin

In [ ]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

## Examples with Sets models

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization


In [ ]:
batch_size=32

train_ds = keras.utils.text_dataset_from_directory('aclImdb/train', batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory('aclImdb/val', batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory('aclImdb/test', batch_size=batch_size)



Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"I really enjoyed the first episode and am looking forward to more. A little soft on the crime front (it's almost an afterthought and not terribly suspenseful or fleshed out) but thought the romance angle was wonderfully charming. Will be watching again for sure! <br /><br />I'm hoping that they'll have a bigger role for the aunts who are wonderful actresses and were somewhat underused this time around. The actress who plays the assistant/waitress (also from Bewitched I think) is very sweet and bubbly and comes off as nicely dorky and sweet instead of dumb and annoying which is very nice.<br /><br />Check it out.", shape=(), dtype=string)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


In [ ]:
txt_vec = TextVectorization(
    max_tokens=20000,
    output_mode="multi_hot"
)

text_only_train_ds = train_ds.map(lambda x, y: x)
txt_vec.adapt(text_only_train_ds)

<MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>

In [ ]:
binary_1gram_train_ds = train_ds.map(lambda x, y: (txt_vec(x), y), num_parallel_calls=12)
binary_1gram_val_ds = val_ds.map(lambda x, y: (txt_vec(x), y), num_parallel_calls=12)
binary_1gram_test_ds = test_ds.map(lambda x, y: (txt_vec(x), y), num_parallel_calls=12)


In [ ]:
for inputs, targets in binary_1gram_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(max_tokens=20000, hidden_dim=16):
  inputs = keras.Input(shape=(max_tokens,))
  x = layers.Dense(hidden_dim, activation="relu")(inputs)
  x = layers.Dropout(0.5)(x)
  outputs = layers.Dense(1, activation="sigmoid")(x)
  model = keras.Model(inputs, outputs)
  model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])

  return model
  

In [ ]:
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.keras",
                                    save_best_only=True)
]

model.fit(
    binary_1gram_train_ds.cache(), 
    validation_data=binary_1gram_val_ds.cache(), 
    epochs=10,
    callbacks=callbacks
)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 15s 23ms/step - loss: 0.4048 - accuracy: 0.8322 - val_loss: 0.2875 - val_accuracy: 0.8952
Epoch 2/10
625/625 [=======================

In [ ]:
model = keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

782/782 [==============================] - 10s 12ms/step - loss: 0.2982 - accuracy: 0.8808
Test acc: 0.881


### Test with 2grams

In [ ]:
txt_vec = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="multi_hot"
)

# Need to call adapt before using it
txt_vec.adapt(text_only_train_ds)

binary_2gram_train_ds = train_ds.map(
    lambda x, y: (txt_vec(x), y),
    num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(
    lambda x, y: (txt_vec(x), y),
    num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(
    lambda x, y: (txt_vec(x), y),
    num_parallel_calls=4)

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_2gram.keras",
                                    save_best_only=True)
]
model.fit(binary_2gram_train_ds.cache(),
          validation_data=binary_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_2 (Dense)             (None, 16)                320016    
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 13s 20ms/step - loss: 0.3750 - accuracy: 0.8461 - val_loss: 0.2711 - val_accuracy: 0.8994
Epoch 2/10
625/625 [=====================

In [ ]:

model = keras.models.load_model("binary_2gram.keras")
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

782/782 [==============================] - 11s 14ms/step - loss: 0.2747 - accuracy: 0.8990
Test acc: 0.899


### Test with TF-IDF

In [ ]:
txt_vec = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="tf_idf",
)

txt_vec.adapt(text_only_train_ds)

tfidf_2gram_train_ds = train_ds.map(
    lambda x, y: (txt_vec(x), y),
    num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(
    lambda x, y: (txt_vec(x), y),
    num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(
    lambda x, y: (txt_vec(x), y),
    num_parallel_calls=4)

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2gram.keras",
                                    save_best_only=True)
]
model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_4 (Dense)             (None, 16)                320016    
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 14s 22ms/step - loss: 0.4908 - accuracy: 0.7764 - val_loss: 0.2951 - val_accuracy: 0.8962
Epoch 2/10
625/625 [=====================

In [ ]:

model = keras.models.load_model("tfidf_2gram.keras")
print(f"Test acc: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

782/782 [==============================] - 12s 15ms/step - loss: 0.2973 - accuracy: 0.8919
Test acc: 0.892


### Making predictions

In [ ]:
import tensorflow as tf

inputs = keras.Input(shape=(1,), dtype="string")
processed_inputs = txt_vec(inputs)
outputs = model(processed_inputs)
inference_model = keras.Model(inputs, outputs)

raw_text_data = tf.convert_to_tensor([
    ["That was an excellent movie, I loved it."],
])
predictions = inference_model(raw_text_data)
print(f"{float(predictions[0] * 100):.2f} percent positive")

81.82 percent positive


## Examples with Sequence models